In [1]:
import cv2
import json
import os
import re
import pandas as pd
from pathlib import Path
import glob
import torch
from transformers import AutoTokenizer,BertForSequenceClassification,AdamW, BertTokenizer,AutoModelForSequenceClassification,DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizerFast


/home/ocr_gpt4_syrua/doc-ocr/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!wget https://guillaumejaume.github.io/FUNSD/dataset.zip -O dataset.zip

--2024-09-17 13:39:21--  https://guillaumejaume.github.io/FUNSD/dataset.zip
Resolving guillaumejaume.github.io (guillaumejaume.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to guillaumejaume.github.io (guillaumejaume.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16838830 (16M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  16.06M  5.74MB/s    in 2.8s    

2024-09-17 13:39:25 (5.74 MB/s) - ‘dataset.zip’ saved [16838830/16838830]



In [5]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/training_data/
  inflating: dataset/training_data/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/dataset/
   creating: __MACOSX/dataset/training_data/
  inflating: __MACOSX/dataset/training_data/._.DS_Store  
   creating: dataset/training_data/images/
  inflating: dataset/training_data/images/92091873.png  
   creating: __MACOSX/dataset/training_data/images/
  inflating: __MACOSX/dataset/training_data/images/._92091873.png  
  inflating: dataset/training_data/images/91939637.png  
  inflating: __MACOSX/dataset/training_data/images/._91939637.png  
  inflating: dataset/training_data/images/87533049.png  
  inflating: __MACOSX/dataset/training_data/images/._87533049.png  
  inflating: dataset/training_data/images/01073843.png  
  inflating: __MACOSX/dataset/training_data/images/._01073843.png  
  inflating: dataset/training_data/images/92586242.png  
  inflating: __MACOSX/dataset/training_data/images/._92586242.pn

In [2]:
class Dataset():
    def __init__(self, path_anot):
        self.path_anot= path_anot

    def __iter__(self):
      with os.scandir(self.path_anot) as files:
        for file in files:
          yield file.name
    def __len__(self):
      i=0
      for file in self:
        i+=1
      return i
    def listTextLabel(self, o):
      anot=json.loads(open(self.path_anot+'/'+o).read())
      txt=[]
      lbl=[]
      for block in list(anot.values())[0]:
        txt.append(block['text'])
        lbl.append(block['label'])
      return (txt,lbl)
    def prep(self):
      dic={'question':0, 'answer':1, 'header':2, 'other':3}
      text=[]
      labels=[]
      for file in self:
        txt,lbl=self.listTextLabel(file)
        text=text+txt
        labels=labels+lbl
      return (text,[dic[x] for x in labels])

In [3]:
dataset_train=Dataset('dataset/training_data/annotations')
dataset_test=Dataset('dataset/testing_data/annotations')

In [4]:
len(dataset_train)


149

In [5]:
text_train, labels_train=dataset_train.prep()
text_test, labels_test=dataset_test.prep()

In [11]:
len(labels_test)

2332